<a href="https://colab.research.google.com/github/RaulMrSouza/Relatorio-Covid-por-Estado/blob/master/Relatorio_Covid_por_Estado.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
#pacotes e imports iniciais
!pip install folium==0.8.2
import folium
from folium import plugins
import pandas as pd
import json
!pip install requests==v2.21.0
import requests
import io
import plotly
import plotly.offline as py
import plotly.graph_objs as go
import warnings
warnings.filterwarnings('ignore')

In [0]:
#@title Usar Runtime->Run (Ctrl+F10) after para atualizar relatório { display-mode: "form", run: "auto"  }

Estado = 'SP' #@param ["AC", "AL", "AM", "AP", "BA", "CE", "DF", "ES", "GO", "MA", "MT", "MS", "MG", "PA", "PB", "PR", "PE", "PI", "RJ", "RN", "RO", "RS", "RR", "SC", "SE", "SP", "TO"]

top_10_cidades_em = "Casos Confirmados por 100k habitantes" #@param [ "Casos Confirmados por 100k habitantes", "Total de Confirmados",  "Total de Mortes", "Taxa de Mortalidade"]
Mapa_coropletico_por = "Casos Confirmados por 100k habitantes" #@param [ "Casos Confirmados por 100k habitantes", "Total de Confirmados",  "Total de Mortes", "Taxa de Mortalidade"]
Grafico_de_barras_por = "Mortes por dia" #@param ["Mortes por dia", "Novos Confirmados por dia", "Casos Confirmados por 100k habitantes", "Total de Confirmados",  "Total de Mortes", "Taxa de Mortalidade"]

In [0]:

#baixar csv do dataset
url="https://brasil.io/dataset/covid19/caso_full/?state="+Estado+"&is_last=true&place_type=city&format=csv"
response = requests.get(url)
file_object = io.StringIO(response.content.decode('utf-8'))
covid = pd.read_csv(file_object)

url="https://brasil.io/dataset/covid19/caso_full/?state="+Estado+"&place_type=state&format=csv"
response = requests.get(url)
file_object = io.StringIO(response.content.decode('utf-8'))
estado = pd.read_csv(file_object)

covid = covid.rename(columns={"last_available_confirmed_per_100k_inhabitants": "Casos Confirmados por 100k habitantes", "last_available_confirmed": "Total de Confirmados", "last_available_deaths": "Total de Mortes"
                    , "estimated_population_2019": "População Estimada em 2019", "city" : "Nome", "new_deaths": "Mortes por dia", "last_available_death_rate": "Taxa de Mortalidade", "new_confirmed": "Novos Confirmados por dia"})

estado = estado.rename(columns={"last_available_confirmed_per_100k_inhabitants": "Casos Confirmados por 100k habitantes", "last_available_confirmed": "Total de Confirmados", "last_available_deaths": "Total de Mortes"
                    , "estimated_population_2019": "População Estimada em 2019", "city" : "Nome", "new_deaths": "Mortes por dia", "last_available_death_rate": "Taxa de Mortalidade", "new_confirmed": "Novos Confirmados por dia"})
#remover nulos
covid = covid[covid['Casos Confirmados por 100k habitantes'] > 0]
idEstado = int(estado['city_ibge_code'].iloc[0])

In [0]:
#trazer json de malha geografica da api do Ibge

headers = {
    'Content-Type': 'application/json;charset=UTF-8', 'User-Agent': 'google-colab',
    'Accept': 'application/json, text/plain, */*', 'Accept-Encoding': 'gzip, deflate, br', 'Accept-Language': 'pt-BR,pt;q=0.9,en-US;q=0.8,en;q=0.7'
}
cidades = requests.get("https://servicodados.ibge.gov.br/api/v2/malhas/"+str(idEstado)+"/?formato=application/vnd.geo+json&resolucao=5", headers=headers)
cidades = cidades.json()

In [0]:
covid['codarea'] = covid['city_ibge_code']
covid = covid.set_index('codarea')
covid['codarea'] = covid.index.astype(int).astype(str)

In [0]:
covid = covid.sort_values(by=[top_10_cidades_em], ascending=False)
covid[['Nome', 'Casos Confirmados por 100k habitantes', 'Total de Confirmados', 'Total de Mortes', 'Taxa de Mortalidade', 'População Estimada em 2019']].head(10)
#.style.set_caption('10 cidades com maior '+Mapa)

,Nome,Casos Confirmados por 100k habitantes,Total de Confirmados,Total de Mortes,Taxa de Mortalidade,População Estimada em 2019
codarea,,,,,,
3524501.0,Jaci,1117.87180,79,2,0.0253,7067.0
3520202.0,Igaratá,1059.36648,101,1,0.0099,9534.0
3548500.0,Santos,1048.20787,4542,188,0.0414,433311.0
3547650.0,Santa Salete,841.42395,13,0,0.0000,1545.0
3550308.0,São Paulo,610.47878,74796,4920,0.0658,12252023.0
3536208.0,Pariquera-Açu,580.21173,114,2,0.0175,19648.0
3518701.0,Guarujá,570.74384,1829,76,0.0416,320459.0
3513504.0,Cubatão,559.27470,731,47,0.0643,130705.0
3548807.0,São Caetano do Sul,538.70549,868,47,0.0541,161127.0


In [0]:
estado = estado.sort_values(by=['date'])
estado = estado.tail(90)

In [0]:
estado = estado.set_index('date')
estado['date'] = estado.index
estado['Media 7 dias'] = estado[Grafico_de_barras_por].rolling(window=7).mean()

In [0]:
from plotly.graph_objs.layout.shape import Line
fig = go.Figure(
    data=[go.Bar(y=estado[Grafico_de_barras_por], x=estado.index, name=Grafico_de_barras_por)
    , go.Line(y=estado['Media 7 dias'], x=estado.index, name="Media 7 dias")
    ],
    layout_title_text=Grafico_de_barras_por+" com Média de 7 dias"
)
fig.show()

In [0]:
#adicionar propriedades no geojson
maxLatitude = -200
maxLongitude = -200
minLatitude = 200
minLongitude = 200
#adicionar campos para layout no mapa para cada cidade
for cidade in cidades['features']: 
  codarea = int(cidade['properties']['codarea'])
  if codarea in covid.index.values:
    cidade['properties']['Cidade'] = str(covid.loc[codarea, "Nome"])
    cidade['properties']['Data Informacao'] = covid.loc[codarea, "date"]
    cidade['properties']['Populacao'] = int(covid.loc[codarea, "População Estimada em 2019"])
    cidade['properties']['Casos por 100k Habitantes'] = int(covid.loc[codarea, "Casos Confirmados por 100k habitantes"])
    cidade['properties']['Total de Confirmados'] = int(covid.loc[codarea, "Total de Confirmados"])
    cidade['properties']['Total de mortes'] = int(covid.loc[codarea, "Total de Mortes"])
    cidade['properties']['Taxa de mortalidade'] = float(covid.loc[codarea, "Taxa de Mortalidade"])
  else:
    cidade['properties']['Cidade'] = 'Sem Dados'
    cidade['properties']['Data Informacao'] = ''
    cidade['properties']['Populacao'] = 0
    cidade['properties']['Casos por 100k Habitantes'] = 0
    cidade['properties']['Total de Confirmados'] = 0
    cidade['properties']['Total de mortes'] = 0
    cidade['properties']['Taxa de mortalidade'] = 0
  maxLongitude = max(cidade['properties']['centroide'][0], maxLongitude)
  maxLatitude = max(cidade['properties']['centroide'][1], maxLatitude)
  minLongitude = min(cidade['properties']['centroide'][0], minLongitude)
  minLatitude = min(cidade['properties']['centroide'][1], minLatitude)

In [0]:
#mapa com casos por cidade
m = folium.Map(
    location=[(maxLatitude + minLatitude)/2, (maxLongitude + minLongitude)/2]
)

m.fit_bounds([[minLatitude, minLongitude], [maxLatitude, maxLongitude]]) 

c = folium.Choropleth(
    geo_data = cidades, 
    data = covid,
    columns = ['codarea', Mapa_coropletico_por],
    key_on = 'feature.properties.codarea',
    bins = 4,
    fill_color = 'YlOrRd', 
    fill_opacity = 0.7, 
    line_opacity = 0.3,
    legend_name = Mapa_coropletico_por
).add_to(m)

c.geojson.add_child(
    folium.features.GeoJsonTooltip(fields=['Cidade', 'Data Informacao', 'Casos por 100k Habitantes', 'Populacao', 'Total de Confirmados', 'Total de mortes', 'Taxa de mortalidade'], labels=True)
)

folium.LayerControl().add_to(m)

display(m)